In [277]:
import numpy as np
import json
import os
import collections
import csv

In [278]:
# Jsonファイルの読み込み

filename = "dataset_my-task_2021-09-16_08-10-06-950.json"
dir_name = "log/"

with open(filename, "r") as f:
    json_obj = json.load(f)

In [279]:
# review のtextとrating / wordのリスト化

word_2d_list = []
rating_list = []
text_list = []
reviewer_loc_list = []
reviewers_cnt_list= []
is_funny_cnt_list = []
is_useful_cnt_list = []
is_cool_cnt_list = []
avoid_word_list = ["(", ")", "!", ",", ":", ";" ,"?", ".", "+", "--", "\"", "/", "#"]


for i in range(len(json_obj[0]["reviews"])):
    word_list = json_obj[0]["reviews"][i]["text"].split()
    text_list.append(json_obj[0]["reviews"][i]["text"])
    for avoid_word in avoid_word_list:
        word_list = [ x.replace(avoid_word, "").lower() for x in word_list]
    word_list = [x for x in word_list if not x.isdigit() and x != "" and len(x) < 20 and len(x) != 0]
    word_2d_list.append(word_list)
    rating_list.append(int(json_obj[0]["reviews"][i]["rating"]))
    reviewer_loc_list.append(json_obj[0]["reviews"][i]["reviewerLocation"])
    reviewers_cnt_list.append(json_obj[0]["reviews"][i]["reviewerReviewCount"])
    is_cool_cnt_list.append(json_obj[0]["reviews"][i]["isCoolCount"])
    is_funny_cnt_list.append(json_obj[0]["reviews"][i]["isFunnyCount"])
    is_useful_cnt_list.append(json_obj[0]["reviews"][i]["isUsefulCount"])
# print(json_obj[0]["reviews"][0]["text"])

In [280]:
# wordを集計

word_cnt = collections.Counter()

for word_list, rating in zip(word_2d_list, rating_list):
    rating_val = rating - 3
    for word in word_list:
        word_cnt[word.lower()] += rating_val

In [281]:
# 星毎に、星の数を集計

word_dict = word_cnt.copy()
# print(word_dict)
for k, v in word_dict.items():
    word_dict[k] = list([v, 0 ,0, 0, 0, 0 ])
#     print(word_dict[k])

for word_list, rating in zip(word_2d_list, rating_list):
    for word in word_list:
        word_dict[word.lower()][rating ] += 1
#         if len(word_dict[word.lower()]) <= 6: # 例文をappend済みでなければ
#             word_dict[word.lower()].append(word_list)


In [282]:
# review textをword毎に集計

output_example_wordlist = ["ramen", "tantanmen"]

for output_example in output_example_wordlist:
    if os.path.exists(dir_name + output_example.lower() + "_check_" + ".txt"):
        continue
    else:
        with open(dir_name + output_example.lower() + "_check_" + ".txt", "w") as f:
            pass
        for word_list, text, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt in zip(word_2d_list, text_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list ):
            for word in word_list:
                if word == output_example:
                    with open(dir_name + word.lower() + "_rating_" + str(rating) + ".txt", "a") as f:
                        f.writelines("reviewer Location : " + reviewer_loc + "\n")
                        f.writelines("Reviewer Count : " + str(reviewers_cnt) + "\n")
                        f.writelines("is Funny Count : " + str(is_funny_cnt) + "\n")
                        f.writelines("is Usuful Count : " + str(is_useful_cnt) + "\n")
                        f.writelines("is Cool Count : " + str(is_cool_cnt) + "\n\n")
                        f.writelines(text)
                        f.writelines("\n\n===========================================\n\n")
                    break

In [283]:
# 全ワードのtext列挙

for word_d in word_dict.keys():
    if os.path.exists(dir_name + word_d.lower() + "_check_" + ".txt"):
        continue
    else:
        with open(dir_name + word_d.lower() + "_check_" + ".txt", "w") as f:
            pass 
    for word_list, text, rating, reviewer_loc, reviewers_cnt, is_cool_cnt, is_funny_cnt, is_useful_cnt in zip(word_2d_list, text_list, rating_list, reviewer_loc_list, reviewers_cnt_list, is_cool_cnt_list, is_funny_cnt_list, is_useful_cnt_list ):
        for word in word_list:
            if word == word_d:
                with open(dir_name + word.lower() + "_rating_" + str(rating) + ".txt", "a") as f:
                    f.writelines("reviewer Location : " + reviewer_loc + "\n")
                    f.writelines("Reviewer Count : " + str(reviewers_cnt) + "\n")
                    f.writelines("is Funny Count : " + str(is_funny_cnt) + "\n")
                    f.writelines("is Usuful Count : " + str(is_useful_cnt) + "\n")
                    f.writelines("is Cool Count : " + str(is_cool_cnt) + "\n\n")
                    f.writelines(text)
                    f.writelines("\n\n===========================================\n\n")
                break

In [275]:
# sortしてファイルに出力

output_file_name = dir_name + "output.csv"

word_dict_sorted = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)

with open(output_file_name, "w") as f:
    writer = csv.writer(f)
    writer.writerow(["word", "review総計" , "星1", "星2", "星3", "星4", "星5"])
    for data_list in word_dict_sorted:
        writer.writerow([ data_list[0], data_list[1][0], data_list[1][1], data_list[1][2], data_list[1][3], data_list[1][4], data_list[1][5] ])